# Pandas (part 2)

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt #Vẽ chart

In [3]:
# Load data
data_path = 'C:\\Users\\duyph\\Documents\\DAP2308R1\\data\\'
data_name = 'output_OnlineRetail.csv'
df = pd.read_csv(data_path + data_name, encoding= 'unicode_escape')

In [4]:
# Head
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,GoodType,Year,Month,Revenue,Date
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6000,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,Regular,2010,12,15300.0,1
1,536365,71053,WHITE METAL LANTERN,6000,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,Regular,2010,12,20340.0,1
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8000,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,Regular,2010,12,22000.0,1
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6000,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,Regular,2010,12,20340.0,1
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6000,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,Regular,2010,12,20340.0,1


In [5]:
df.shape

(541909, 13)

## 2. Thống kê mô tả

- Doanh thu thấp nhất trên data

In [6]:
df["Revenue"].min()

-168469600.0

- Kiểm tra xem kết quả âm là do đâu

In [7]:
# Lọc ra những dòng có revenue < 0
df.loc[df["Revenue"] < 0, :].head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,GoodType,Year,Month,Revenue,Date
141,C536379,D,Discount,-1000,2010-12-01 09:41:00,27.50,14527.0,United Kingdom,Luxury,2010,12,-27500.0,1
154,C536383,35004C,SET OF 3 COLOURED FLYING DUCKS,-1000,2010-12-01 09:49:00,4.65,15311.0,United Kingdom,Regular,2010,12,-4650.0,1
235,C536391,22556,PLASTERS IN TIN CIRCUS PARADE,-12000,2010-12-01 10:24:00,1.65,17548.0,United Kingdom,Regular,2010,12,-19800.0,1
236,C536391,21984,PACK OF 12 PINK PAISLEY TISSUES,-24000,2010-12-01 10:24:00,0.29,17548.0,United Kingdom,Cheap,2010,12,-6960.0,1
237,C536391,21983,PACK OF 12 BLUE PAISLEY TISSUES,-24000,2010-12-01 10:24:00,0.29,17548.0,United Kingdom,Cheap,2010,12,-6960.0,1


- Nguyên nhân: quantity có thể âm 
    - Quantity < 0: công ty nhập hàng vào
    - Quantity > 0: công ty bán hàng ra

- Do vậy cần lọc ra những hóa đơn liên quan đến BÁN HÀNG

In [10]:
df_sell = df.loc[df["Quantity"] > 0, :].copy() 
df_sell.shape

(531285, 13)

In [11]:
df_sell["Quantity"].min()

1000

- Tính lại giá trị hóa đơn nhỏ nhất

In [12]:
df_sell["Revenue"].min()

-11062060.0

In [13]:
df_sell["UnitPrice"].min()

-11062.06

- Chúng ta cần làm sạch data bằng cách loại bỏ UnitPrice < 0

In [14]:
# Đọc lại data gốc:
df = pd.read_csv(data_path + data_name, encoding= 'unicode_escape')

# Loại bỏ UnitPrice < 0 và lưu vào 1 dataframe khác
df_true_price = df.loc[df["UnitPrice"] >= 0, :].copy()

- Tính giá trị hóa đơn cao nhất

In [15]:
df_true_price["Revenue"].max()

168469600.0

- Tính doanh thu trung bình trên mỗi hóa đơn

In [16]:
df_true_price["Revenue"].mean()

18028.68767888217

- Tính median doanh thu trên mỗi hóa đơn

In [17]:
df_true_price["Revenue"].median()

9750.0

## 3. Group by and aggregation

- Chỉ sử dụng group by cho những cột có unique values nhỏ
- Luôn luôn đặt aggregration ngay sau group by

### 3.1. Examples with group by (1 column)

In [18]:
# Mean quantity by country
df_true_price.groupby("Country")["Quantity"].mean().nlargest(10)

Country
Netherlands       84406.579502
Sweden            77136.363636
Japan             70441.340782
Australia         66444.003177
Singapore         22855.895197
Denmark           21048.843188
Czech Republic    19733.333333
Lithuania         18628.571429
Canada            18298.013245
Norway            17722.836096
Name: Quantity, dtype: float64

In [19]:
# Median quantity by country
df_true_price.groupby("Country")["Quantity"].median().nlargest(10)

Country
Netherlands       72000.0
Japan             36000.0
Australia         24000.0
Czech Republic    24000.0
Sweden            20000.0
Lithuania         16000.0
Canada            12000.0
Denmark           12000.0
Iceland           12000.0
Norway            12000.0
Name: Quantity, dtype: float64

Note: results are `Series`

In [20]:
# Save the aggregation to result
result = df_true_price.groupby("Country")["Quantity"].median().nlargest(10)

In [21]:
# Verify the type
type(result)

pandas.core.series.Series

In [22]:
# Access Male index
result.loc["Netherlands"]

72000.0

In [23]:
# Or just like this
result["Netherlands"]

72000.0

- More than 1 summary statistics (dùng `.agg`)

In [24]:
# Mean and median bill by sex
df_true_price.groupby("Country")["Quantity"].agg(["min", "mean", "median", "max",  "std"])

,min,mean,median,max,std
Country,,,,,
Australia,-120000,66444.003177,24000.0,1152000,97686.932221
Austria,-48000,12037.406484,9000.0,288000,21745.484526
Bahrain,-54000,13684.210526,6000.0,96000,30016.759451
Belgium,-12000,11189.946834,10000.0,272000,13601.441194
Brazil,2000,11125.000000,10000.0,24000,8476.723347
Canada,1000,18298.013245,12000.0,504000,46682.587004
Channel Islands,-2000,12505.277045,10000.0,407000,22570.877031
Cyprus,-33000,10155.948553,5000.0,288000,23263.590490
Czech Republic,-24000,19733.333333,24000.0,72000,22813.990465


In [25]:
# A shortcut
df_true_price.groupby("Country")["Quantity"].describe()

,count,mean,std,min,25%,50%,75%,max
Country,,,,,,,,
Australia,1259.0,66444.003177,97686.932221,-120000.0,6000.0,24000.0,96000.0,1152000.0
Austria,401.0,12037.406484,21745.484526,-48000.0,6000.0,9000.0,12000.0,288000.0
Bahrain,19.0,13684.210526,30016.759451,-54000.0,5000.0,6000.0,10000.0,96000.0
Belgium,2069.0,11189.946834,13601.441194,-12000.0,4000.0,10000.0,12000.0,272000.0
Brazil,32.0,11125.000000,8476.723347,2000.0,3000.0,10000.0,18000.0,24000.0
Canada,151.0,18298.013245,46682.587004,1000.0,6000.0,12000.0,20000.0,504000.0
Channel Islands,758.0,12505.277045,22570.877031,-2000.0,4000.0,10000.0,12000.0,407000.0
Cyprus,622.0,10155.948553,23263.590490,-33000.0,2000.0,5000.0,12000.0,288000.0
Czech Republic,30.0,19733.333333,22813.990465,-24000.0,12000.0,24000.0,24000.0,72000.0


In [26]:
df_true_price["Quantity"].describe()['75%']

10000.0

- Reset index để tiếp tục tính toán (nếu có)

In [27]:
df_true_price.groupby("Country")["Quantity"].describe().reset_index()

,Country,count,mean,std,min,25%,50%,75%,max
0,Australia,1259.0,66444.003177,97686.932221,-120000.0,6000.0,24000.0,96000.0,1152000.0
1,Austria,401.0,12037.406484,21745.484526,-48000.0,6000.0,9000.0,12000.0,288000.0
2,Bahrain,19.0,13684.210526,30016.759451,-54000.0,5000.0,6000.0,10000.0,96000.0
3,Belgium,2069.0,11189.946834,13601.441194,-12000.0,4000.0,10000.0,12000.0,272000.0
4,Brazil,32.0,11125.000000,8476.723347,2000.0,3000.0,10000.0,18000.0,24000.0
5,Canada,151.0,18298.013245,46682.587004,1000.0,6000.0,12000.0,20000.0,504000.0
6,Channel Islands,758.0,12505.277045,22570.877031,-2000.0,4000.0,10000.0,12000.0,407000.0
7,Cyprus,622.0,10155.948553,23263.590490,-33000.0,2000.0,5000.0,12000.0,288000.0
8,Czech Republic,30.0,19733.333333,22813.990465,-24000.0,12000.0,24000.0,24000.0,72000.0
9,Denmark,389.0,21048.843188,27402.501578,-25000.0,12000.0,12000.0,24000.0,256000.0


### 3.2. Examples with group by (multiple columns)

### .groupby([])[].<aggregate nào đó>

In [28]:
# Mean Quantity by Country and GoodType
df_true_price.groupby(["Country", "GoodType"])["Quantity"].mean()

Country         GoodType
Australia       Cheap       124671.936759
                Luxury       22590.909091
                Regular      52453.252033
Austria         Cheap        22447.058824
                Luxury        2896.551724
                                ...      
United Kingdom  Luxury        2216.713817
                Regular       7161.148615
Unspecified     Cheap        14034.482759
                Luxury        1230.769231
                Regular       5223.974763
Name: Quantity, Length: 110, dtype: float64

- Note: Kết quả phía trên là một multi-index Series (2 levels)

In [29]:
# Lưu vào resulbt
result = df_true_price.groupby(["Country", "GoodType"])["Quantity"].mean()

In [33]:
result

Country         GoodType
Australia       Cheap       124671.936759
                Luxury       22590.909091
                Regular      52453.252033
Austria         Cheap        22447.058824
                Luxury        2896.551724
                                ...      
United Kingdom  Luxury        2216.713817
                Regular       7161.148615
Unspecified     Cheap        14034.482759
                Luxury        1230.769231
                Regular       5223.974763
Name: Quantity, Length: 110, dtype: float64

In [30]:
# Mean quantity for the goods that Cheap and in United Kingdom
result[("United Kingdom", "Cheap")]

15085.426942786404

In [31]:
# Mean quantity for the goods that Luxury and in Singapore
result[("Singapore", "Luxury")]

1736.842105263158

- Ta cũng có thể reset index để được kết quả là 1 data frame

In [34]:
result.reset_index()

,Country,GoodType,Quantity
0,Australia,Cheap,124671.936759
1,Australia,Luxury,22590.909091
2,Australia,Regular,52453.252033
3,Austria,Cheap,22447.058824
4,Austria,Luxury,2896.551724
...,...,...,...
105,United Kingdom,Luxury,2216.713817
106,United Kingdom,Regular,7161.148615
107,Unspecified,Cheap,14034.482759
108,Unspecified,Luxury,1230.769231


- Tương tự như group by 1 column, ta cũng có thể dùng `.describe()`

In [35]:
df_true_price.groupby(["Country", "GoodType"])["Quantity"].describe()

count           mean            std         min  \
Country        GoodType                                                       
Australia      Cheap        253.0  124671.936759  143144.612342    -36000.0   
               Luxury        22.0   22590.909091   25045.975042     -1000.0   
               Regular      984.0   52453.252033   76383.821246   -120000.0   
Austria        Cheap         85.0   22447.058824   39509.644959    -48000.0   
               Luxury        29.0    2896.551724    1915.068585      1000.0   
...                           ...            ...            ...         ...   
United Kingdom Luxury     22269.0    2216.713817    7981.662732   -150000.0   
               Regular   369001.0    7161.148615  257364.280962 -80995000.0   
Unspecified    Cheap        116.0   14034.482759   12075.797595      1000.0   
               Luxury        13.0    1230.769231     438.529010      1000.0   
               Regular      317.0    5223.974763    5714.487970      1000.0   

                             25%      50%       75%         max  
Country        GoodType                                          
Australia      Cheap     16000.0  96000.0  200000.0   1152000.0  
               Luxury     4500.0  16000.0   32000.0    100000.0  
               Regular    6000.0  24000.0   80000.0    960000.0  
Austria        Cheap     12000.0  12000.0   24000.0    288000.0  
               Luxury     2000.0   2000.0    4000.0      9000.0  
...                          ...      ...       ...         ...  
United Kingdom Luxury     1000.0   1000.0    2000.0    300000.0  
               Regular    1000.0   3000.0    6000.0  80995000.0  
Unspecified    Cheap      2000.0  12000.0   24000.0     48000.0  
               Luxury     1000.0   1000.0    1000.0      2000.0  
               Regular    1000.0   2000.0    7000.0     24000.0  

[110 rows x 8 columns]

- Reset index

In [36]:
df_true_price.groupby(["Country", "GoodType"])["Quantity"].describe().reset_index()

,Country,GoodType,count,mean,std,min,25%,50%,75%,max
0,Australia,Cheap,253.0,124671.936759,143144.612342,-36000.0,16000.0,96000.0,200000.0,1152000.0
1,Australia,Luxury,22.0,22590.909091,25045.975042,-1000.0,4500.0,16000.0,32000.0,100000.0
2,Australia,Regular,984.0,52453.252033,76383.821246,-120000.0,6000.0,24000.0,80000.0,960000.0
3,Austria,Cheap,85.0,22447.058824,39509.644959,-48000.0,12000.0,12000.0,24000.0,288000.0
4,Austria,Luxury,29.0,2896.551724,1915.068585,1000.0,2000.0,2000.0,4000.0,9000.0
...,...,...,...,...,...,...,...,...,...,...
105,United Kingdom,Luxury,22269.0,2216.713817,7981.662732,-150000.0,1000.0,1000.0,2000.0,300000.0
106,United Kingdom,Regular,369001.0,7161.148615,257364.280962,-80995000.0,1000.0,3000.0,6000.0,80995000.0
107,Unspecified,Cheap,116.0,14034.482759,12075.797595,1000.0,2000.0,12000.0,24000.0,48000.0
108,Unspecified,Luxury,13.0,1230.769231,438.529010,1000.0,1000.0,1000.0,1000.0,2000.0


## 4. Một số thao tác khác với columns

### 4.1 Đổi tên cột

In [38]:
# Preview
df_true_price.head(1)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,GoodType,Year,Month,Revenue,Date
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6000,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,Regular,2010,12,15300.0,1


In [39]:
# Tạo df2 gồm 3 cột đầu
df2 = df_true_price.iloc[:, :3]
df2.head(1)

,InvoiceNo,StockCode,Description
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER


- Đổi tên cột `StockCode` thành `stock_code` và `InvoiceNo` thành `invoice_no` (cách 1)
- Cách này phù hợp cho sửa tên một số ít cột

In [40]:
df2.rename(columns={"StockCode": "stock_code", "InvoiceNo": "invoice_no"}, inplace=True)
df2.head(2)

,invoice_no,stock_code,Description
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER
1,536365,71053,WHITE METAL LANTERN


- Đổi tên tất cả 3 cột thành `stock_code`, `invoice_no`, và `desc`
- Cách này phù hợp khi đổi tên hết tất cả các cột (lưu ý giữ đúng thứ tự)

In [41]:
df2.columns = ["stock_code", "invoice_no", "desc"]
df2.head(1)

,stock_code,invoice_no,desc
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER


## 5. Ép kiểu dữ liệu cột

In [42]:
# Preview
df_true_price.head(1)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,GoodType,Year,Month,Revenue,Date
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6000,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,Regular,2010,12,15300.0,1


In [43]:
# Dtypes
df_true_price.dtypes

InvoiceNo              object
StockCode              object
Description            object
Quantity                int64
InvoiceDate    datetime64[ns]
UnitPrice             float64
CustomerID            float64
Country                object
GoodType               object
Year                    int64
Month                   int64
Revenue               float64
Date                    int64
dtype: object

- Ép cột `Quantity` thành string

In [44]:
df_true_price["Quantity"] = df_true_price["Quantity"].astype(str)
df_true_price.dtypes

InvoiceNo              object
StockCode              object
Description            object
Quantity               object
InvoiceDate    datetime64[ns]
UnitPrice             float64
CustomerID            float64
Country                object
GoodType               object
Year                    int64
Month                   int64
Revenue               float64
Date                    int64
dtype: object

- Ép ngược lại `Quantity` thành int

In [45]:
df_true_price["Quantity"] = df_true_price["Quantity"].astype(int)
df_true_price.dtypes

InvoiceNo              object
StockCode              object
Description            object
Quantity                int32
InvoiceDate    datetime64[ns]
UnitPrice             float64
CustomerID            float64
Country                object
GoodType               object
Year                    int64
Month                   int64
Revenue               float64
Date                    int64
dtype: object

- Ép cột `InvoiceDate` thành datetime

In [37]:
df_true_price["InvoiceDate"] = pd.to_datetime(df_true_price["InvoiceDate"])
df_true_price.dtypes

InvoiceNo              object
StockCode              object
Description            object
Quantity                int64
InvoiceDate    datetime64[ns]
UnitPrice             float64
CustomerID            float64
Country                object
GoodType               object
Year                    int64
Month                   int64
Revenue               float64
Date                    int64
dtype: object